# Model / Class driven events and quest management

Events are happening all the time. Several events happen before the Character quest begins. 

In [3]:
import pandas as pd
import numpy as np
import os

import os
import sys
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()

#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys,pickle
sys.path.append('../')
from lib.builders import people,towns

with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

as a convention I'm adding `e` as an individual event as the result of the DB query. 

In [23]:
class Nation:
    def __init__(self,world,people,origin="kmeans"):
        self.name = self.name_nation(world)
        
        if origin="kmeans":
            #Kmeans is the default (when the world is created)
            self.towns = self.addTowns(world.towns)
        
        
        self.diplomacy = self.add_diplomacy(world)
    
    def add_towns(self,world):
        towns = [town for town in towns 
                 if town.diplomacy.get('nation','none')==self.name]
        return towns
    
    def name_nation(self,world):
        """
        lables a list of nations from a k-means cluster
        """
        nation_name = world.culture.townNameGenerator()
        return nation_name

    def __repr__(self):
        return f"Nation of {self.name}"
       
    def addTowns(self,towns):
        towns = [town for town in towns 
                 if town.diplomacy.get('nation','none')==self.name]
        return towns
        
    def get_capitol(self):
        c = [t for t in self.towns if t.type == 'capitol'][0]
        return c
    
    def get_random_town(self):
        return np.random.choice(self.towns)

    def getRuler_str(self):
        return self.ruler.name
    
    def getRuler(self):
        return self.ruler
    
    def add_diplomacy(self,world):
        # if the world already has nations, then this is a new nation. 
        diplomacy = {}
        otherNations = [n for n in world.nations if n.name != self.name]
        for o in otherNations:
            diplomacy[o] = {'favor':.8}
        return diplomacy
    
    def get_deplomacy_df(self):
        d = pd.DataFrame(self.diplomacy).T.reset_index(drop=False)
        d['nation'] = self.name
        d.columns = ['neighbor','favor','stance','nation']
        return d[['nation','neighbor','favor','stance']]
            
    def appointRuler(self,person):
        t = self.getCapitol_str(self.towns)
        self.ruler = person(f'ruler of {self.name}',t)
        
nation = Nation(world,people)
print(nation)

Nation of Rockcombe


In [25]:
nation.get_deplomacy_df()

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [118]:
for key in possible_events:
    e = get_event(key)
    event = Event(e)
    event.random_assign_agent(world)
    print(event)

bandits have set up camp in the plain of [Nation of Brinehaven]
a noble from [Nation of Magejaw] was married to a noble in [Nation of Fairforth]
raiders from [Nation of Landsplace], plundered [Nation of Landsplace Nation of Magejaw Nation of Landsfellow]
a great temple was founded in the city of town in [Nation of Magejaw]
